
# CAFE vs Perspective — Section-by-Section Runner (Colab)

This notebook lets you run your experiment _step by step_ so you can understand each phase clearly.

**Pipeline**
1. Prepare & augment RealToxicityPrompts (RTP) subset  
2. Train CAFE model  
3. Load Jigsaw evaluation set  
4. Evaluate CAFE vs Perspective API  
5. Generate final research report

> Tip: Run each cell top-to-bottom. If anything fails, re-run the previous cell.


## 1) Environment & GPU

In [1]:
import torch, platform, sys, os, subprocess
print("Python:", sys.version)
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
!nvidia-smi || true

Python: 3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]
PyTorch: 2.8.0+cu126
CUDA available: True
GPU name: Tesla T4
Mon Sep 29 07:54:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             13W /   70W |       2MiB /  15360MiB |      0%      Default |
|                   

## 2) Project Setup


Choose ONE of the following upload/setup methods:

- **A. Google Drive (recommended)**  
  Put your whole project folder in Drive (e.g., `MyDrive/projects/cafe_toxicity/`).  
  Update `PROJECT_DIR` below to that path **after mounting**.

- **B. Upload ZIP**  
  Zip your local project, upload it to Colab, and unzip into `/content/project`.

- **C. Direct upload**  
  Drag & drop files/folders into `/content/project` with the Colab file browser.

> Ensure your project has the `src/` folder and the `run_cafe_vs_perspective.py` file in the project root.


In [2]:
# (Optional) Mount Google Drive
USE_DRIVE = True  # set to False if not using Drive
DRIVE_PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/CAFE-Context-Aware-Fairness-Weighted-Framework-for-Toxicity-Evaluation"

from pathlib import Path

if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# If using ZIP upload instead, uncomment and run these:
# from google.colab import files
# uploaded = files.upload()  # upload your ZIP
# !mkdir -p /content/project
# !unzip -o "*.zip" -d /content/project
# PROJECT_DIR = "/content/project"

# If using Drive:
if USE_DRIVE:
    PROJECT_DIR = DRIVE_PROJECT_PATH

%cd "$PROJECT_DIR"

print("PROJECT_DIR =", PROJECT_DIR)
print("Listing:", os.listdir(PROJECT_DIR))

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/CAFE-Context-Aware-Fairness-Weighted-Framework-for-Toxicity-Evaluation
PROJECT_DIR = /content/drive/MyDrive/Colab Notebooks/CAFE-Context-Aware-Fairness-Weighted-Framework-for-Toxicity-Evaluation
Listing: ['src', 'experiments', 'data', 'requirements.txt', '.env', '.ipynb_checkpoints']


## 3) Install Dependencies

In [3]:
# Adjust below as needed. If you have a requirements.txt in your project, use that.
REQ_TXT = os.path.join(PROJECT_DIR, "requirements.txt")
if os.path.exists(REQ_TXT):
    print("Installing from requirements.txt ...")
    !pip install -q -r "$REQ_TXT"
else:
    print("Installing common packages ...")
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    !pip install -q transformers pandas scikit-learn tqdm matplotlib seaborn sentencepiece accelerate

Installing from requirements.txt ...


## 4) Perspective API Key


Set your API key here. If you skip this, the evaluator will likely fall back to mock scores.

- Create a free key from the Perspective API console.
- **Never** commit secrets to Git.


In [4]:
import os
from google.colab import userdata

os.environ['PERSPECTIVE_API_KEY'] = userdata.get('PERSPECTIVE_API_KEY')
PERSPECTIVE_API_KEY = os.environ['PERSPECTIVE_API_KEY']

print("Has API key:", bool(os.environ.get("PERSPECTIVE_API_KEY")))

Has API key: True


## 5) Import Project Code

In [5]:
import sys, os
from pathlib import Path

# Ensure project root is on sys.path
if str(Path(PROJECT_DIR)) not in sys.path:
    sys.path.insert(0, str(Path(PROJECT_DIR)))
if str(Path(PROJECT_DIR) / "src") not in sys.path:
    sys.path.insert(0, str(Path(PROJECT_DIR) / "experiments"))

print("sys.path entries added.")
# Quick sanity check for expected files
assert os.path.exists(os.path.join(PROJECT_DIR, "experiments", "run_cafe_vs_perspective.py")), "run_cafe_vs_perspective.py not found in project root."
print("Found run_cafe_vs_perspective.py ✔")

sys.path entries added.
Found run_cafe_vs_perspective.py ✔


## 6) Step 1 — Prepare & Augment RTP data

In [6]:

import pandas as pd
from run_cafe_vs_perspective import CAFEExperimentRunner

runner = CAFEExperimentRunner()
# augmented_df = runner.step1_prepare_and_augment_rtp_data()
aug_path = os.path.join(PROJECT_DIR, "data/augmented/rtp_augmented.csv")

augmented_df = pd.read_csv(aug_path)
print("Augmented rows:", len(augmented_df))
augmented_df.head(3)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Augmented rows: 1993


,prompt,continuation,toxicity,severe_toxicity,identity_attack,insult,threat,profanity,sexually_explicit,flirtation,identity_mention,context_label,augmentation_type
0,That movie was absolutely,terrible and boring,0.353677,0.015808,0.001919,0.138106,0.050900,0.525523,0.061390,0.434027,0,0,original
1,Paraphrase: That movie was absolutely terrible...,boring. That movie was absolutely terrible and...,0.353677,0.015808,0.001919,0.138106,0.050900,0.525523,0.061390,0.434027,0,0,paraphrase
2,The politician was accused of,corruption and embezzlement,0.125192,0.056845,0.320248,0.055351,0.011428,0.354903,0.028818,0.179677,1,0,original


## 7) Step 2 — Train CAFE model


> **Note:** Training time depends on GPU and dataset size (the script already subsamples to a manageable size).


In [7]:
cafe_model = runner.step2_train_cafe_model(augmented_df)
print("Model ready.")

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Validating: 100%|██████████| 40/40 [00:02<00:00, 16.64it/s]


Model ready.


In [ ]:
import glob, os, torch
from src.model import CAFEModel  # adjust import to your code

device = "cuda" if torch.cuda.is_available() else "cpu"

# find latest best model
candidates = sorted(
    glob.glob(os.path.join(PROJECT_DIR, "results/models/cafe_*/best_model*.pt"))
)
if not candidates:
    raise FileNotFoundError("No model checkpoint found under results/models/cafe_*/")
model_path = candidates[-1]
print("Loading:", model_path)

# load (weights_only=True is default in PyTorch 2.6, we pass it explicitly)
ckpt = torch.load(model_path, map_location=device, weights_only=True)
model = CAFEModel()               # init with the same config you used for training
model.load_state_dict(ckpt["model_state_dict"])
model.to(device).eval()

## 8) Step 3 — Prepare Jigsaw Dataset

In [9]:

jigsaw_df = runner.step3_prepare_jigsaw_dataset()
# jigsaw_path = os.path.join(PROJECT_DIR, "data/jigsaw/jigsaw_eval.csv")

# jigsaw_df = pd.read_csv(jigsaw_path)
print("Jigsaw rows:", len(jigsaw_df))
jigsaw_df.head(3)

Jigsaw rows: 2000


,comment_text,toxicity,severe_toxicity,identity_attack,insult,threat,profanity,identity_mention,context_label
0,"Yeah right, bad makes perfect sense.",0.538963,0.290751,0.024042,0.257623,0.063288,0.294527,0,1
1,The weather today is absolutely great.,0.037145,0.011449,0.097292,0.030318,0.006081,0.010130,0,0
2,This book about music was fascinating.,0.127321,0.064922,0.048875,0.069468,0.036103,0.032569,0,0


## 9) Step 4 — Evaluate CAFE vs Perspective


If your Perspective API key is set, this will query the live API; otherwise, your implementation may fall back to mock values.

The script evaluates on up to 500 samples for speed.


In [14]:
results = runner.step4_evaluate_models_fixed(cafe_model, jigsaw_df)

✅ Added safe globals for PyTorch loading
STEP 4: Evaluating CAFE vs Perspective API (FIXED)
⚠️ No trained model found, using untrained model
Evaluating on 100 Jigsaw samples...


Analyzing with Perspective API: 100%|██████████| 10/10 [01:39<00:00,  9.93s/it]
/content/drive/MyDrive/Colab Notebooks/CAFE-Context-Aware-Fairness-Weighted-Framework-for-Toxicity-Evaluation/experiments/../src/cafe_vs_perspective.py:287: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax2.set_xticklabels(metrics, rotation=45)


✅ Evaluation completed!


In [15]:
list(results.keys())

['cafe_metrics', 'perspective_metrics', 'raw_predictions', 'comparison']

## 10) Step 5 — Generate Final Research Report

In [17]:
runner.step5_generate_final_research_report(results)
print("Done. Check the results/ folder in your project for models, plots, and reports.")

Done. Check the results/ folder in your project for models, plots, and reports.


## 11) (Optional) Run the Entire Pipeline

In [ ]:
# This will run all steps end-to-end (same as above but in one call).
# Useful once you've verified each section.
runner_full = CAFEExperimentRunner()
runner_full.run_complete_experiment()